**Proyek: Analisis Data Karyawan Fiktif 🧑‍💻 (penerapan data Cleaning and imputation)**


**Skenario:**

**diberikan dataset fiktif employee_data.csv. Dataset ini kacau: ada data yang hilang, tipe data yang salah, dan beberapa outlier. Tugas Anda adalah membersihkan dan menganalisis dataset ini untuk memberikan wawasan kepada departemen SDM.**

---

**Tugas:**

**1.Muat dan Inspeksi Data:**
- Muat data ke dalam DataFrame pandas.
- Gunakan .info() dan .isna().sum() untuk mengidentifikasi masalah awal (nilai hilang, tipe data yang salah).

In [6]:
import pandas as pd

df_employ = pd.read_csv('employee_data.csv')
print(df_employ.head())

print(df_employ.info())
print(df_employ.isna().sum())

   Employee_ID   Department  Years_At_Company  Satisfaction_Score  \
0          101        Sales               5.0                 4.5   
1          102  Engineering               3.0                 3.8   
2          103    Marketing               7.0                 NaN   
3          104        Sales              15.0                 2.1   
4          105  Engineering               NaN                 4.2   

  Last_Salary_USD  
0          85,000  
1          92,000  
2             NaN  
3         150,000  
4          98,000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Employee_ID         10 non-null     int64  
 1   Department          10 non-null     object 
 2   Years_At_Company    9 non-null      float64
 3   Satisfaction_Score  8 non-null      float64
 4   Last_Salary_USD     8 non-null      object 
dtypes: float64(2), i

---

**2.Pembersihan Data:**

- Tangani Data Hilang:

    - Untuk Years_At_Company, imputasi nilai yang hilang dengan median dari kolom tersebut.
    - Untuk Satisfaction_Score, imputasi nilai yang hilang dengan rata-rata (mean) dari kolom tersebut.
    - Untuk Last_Salary_USD, karena ini adalah data penting, hapus baris di mana gaji hilang.

- Perbaiki Tipe Data:

    - Kolom Last_Salary_USD saat ini bertipe object karena ada koma. Hapus koma dan konversikan kolom ini menjadi tipe data numerik (misalnya, float atau int).

---
**3.Analisis dan Rekayasa Fitur:**

- Identifikasi Outlier: Gunakan metode Interquartile Range (IQR) pada kolom Last_Salary_USD untuk mengidentifikasi apakah ada gaji yang dianggap sebagai outlier. Cetak baris data yang merupakan outlier.

- Buat Kolom Baru: Buat kolom baru bernama Employee_Tenure berdasarkan Years_At_Company dengan kategori berikut:
    - "New" (0-3 tahun)
    - "Mid" (4-8 tahun)
    - "Senior" (>8 tahun)

**4.Sajikan Wawasan:**
- Hitung gaji rata-rata untuk setiap Department. Departemen mana yang memiliki gaji rata-rata tertinggi?
- Hitung skor kepuasan rata-rata untuk setiap kategori Employee_Tenure. Apakah karyawan yang lebih lama cenderung lebih puas?